In [1]:
import pandas as pd
import numpy as np

from IPython.display import clear_output, display, HTML
import time
import random
from itertools import count

import pickle

# import scarper components
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

from pymongo import MongoClient

import re

In [2]:
db = MongoClient("mongodb://@52.53.234.224", 27017)

In [14]:
col = pd.DataFrame(db.project.reviews.find())

In [33]:
col[col['title'] == 'So far so good!']

,ASIN,_id,content,pg_num,review_date,review_id,review_score,reviewer_name,title,vine_status
8780,B06XWZWYVP,5d53c329607e5e5c72191c09,Popped it in to my Nintendo Switch and it work...,379,"December 12, 2018",R3OM57OJQAVBUK,5.0,Alex Smith,So far so good!,False
9577,B06XWZWYVP,5d53c4a3607e5e5c72191f26,So far so good. I needed extra storage for my ...,459,"October 4, 2018",R275GNW80BWHRO,5.0,Brandi Dronet,So far so good!,False
15224,B072HRDM55,5d53cf06607e5e5c72193536,Have had for about 6months and works great!,24,"July 2, 2019",RI8JC668MDRUO,5.0,Jon Daley,So far so good!,False
18833,B073JWXGNT,5d53d58b607e5e5c7219434f,It’s working fine so far.,230,"December 10, 2018",R20DYV3XKOCJHQ,5.0,Kao T,So far so good!,False
24875,B073JYC4XM,5d542eb2d2a1c6f67e7bd2dc,No problems so far,197,"January 22, 2019",R25TFX9YZ2KLZ3,5.0,Johnson,So far so good!,False
25080,B073JYC4XM,5d542f11d2a1c6f67e7bd3a9,We love the brand so we decide to get this one...,217,"January 7, 2019",R2QYINQ2C9W76B,5.0,Martha cortes,So far so good!,False
33985,B074RNRM2B,5d544c4f6214cec1254b3d8a,I had a 200gb Sandisk card in my LG V10 and h...,110,"July 22, 2018",R2U0JXX55ENAUK,5.0,LoafTheTabby,So far so good!,False


In [8]:
len(col)

2001

## Creating scraper

In [13]:
# Declaring UserAgent
agent = {'User-Agent': UserAgent().chrome}

# Obtaining Legitimate Cookie
url = 'https://www.google.com/'
session = requests.Session()
cookie = session.get(url, headers = agent).cookies.get_dict()

# proxyDict = { 
#           "http": '18.144.29.167:8888', 
#           "https": '18.144.29.167:8888'
#         }

request = lambda link: requests.Session().get(
                                    link,
                                    headers = agent,
                                    cookies = cookie
#                                     ,proxies= proxyDict
                                )

In [14]:
def review_scrape(ASIN, pg_num):
    # Scrape Reviews
    url = f'https://www.amazon.com/product-reviews/{ASIN}/ref=cm_cr_getr_d_paging_btm_next_{pg_num}?sortBy=recent&pageNumber={pg_num}&formatType=current_format'

    soup = BeautifulSoup(request(url).text)
    
    # Stripping review_id, title and content

    content = [ i.get_text(strip=True) for i in soup.find_all(attrs={'a-size-base review-text review-text-content'}) ]

    _title_and_reviewid = soup.find_all(attrs={'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold'})

    
    get_vine_status = lambda x: (
                                    soup.find_all(attrs={'id': f'customer_review-{x}'})[0]
                                    .find(attrs={'a-link-normal'})
                                    .get_text() == 'VINE VOICE'
                                )

    get_review_score = lambda x: float(
                                            soup.find_all(attrs={'id': f'customer_review-{x}'})[0]
                                            .find(attrs={'a-icon-alt'})
                                            .get_text()
                                            .split(' ')[0]
                                      )

    get_reviewer_name = lambda x: (
                                    soup.find_all(attrs={'id': f'customer_review-{x}'})[0]
                                    .find(attrs={'a-profile-name'})
                                    .get_text()
                                    )


    title = [ i.span.get_text() for i in _title_and_reviewid ]
    review_id = [ i.attrs['href'].split('/')[3] for i in _title_and_reviewid ]
    review_date = [ i.get_text() for i in soup.find_all(attrs={'review-date'})[2:] ]
    vine_status = [ get_vine_status(i) for i in review_id ]
    reviewer_name = [ get_reviewer_name(i) for i in review_id ]
    review_score = [ get_review_score(i) for i in review_id ]

    _ = pd.DataFrame(data= zip(review_id, review_date, review_score, vine_status, reviewer_name, title, content), columns=['review_id', 'review_date', 'review_score', 'vine_status', 'reviewer_name', 'title', 'content'])

    _['ASIN'] = ASIN
    _['pg_num'] = pg_num

    return _.reindex(columns=['ASIN', 'pg_num','review_id', 'review_score', 'vine_status', 'reviewer_name', 'review_date', 'title', 'content'])

In [15]:
review_scrape('B073JWXGNT', 17).head(1)

,ASIN,pg_num,review_id,review_score,vine_status,reviewer_name,review_date,title,content
0,B073JWXGNT,17,R20SINV1YASC8B,5.0,False,Jim Coble,"July 31, 2019",Works great,Works great


## Establish targets SKUs

In [ ]:
col = []

url = 'https://www.amazon.com/Samsung-MicroSDXC-Adapter-MB-ME128GA-AM/dp/B06XWZWYVP/ref=sr_1_5?keywords=sandisk%2Bmicro%2Bsd&qid=1565739330&s=gateway&sr=8-5&th=1'

soup = BeautifulSoup(request(url).text)

_ = []

for i in soup.find_all(attrs={'a-unordered-list a-nostyle a-button-list a-declarative a-button-toggle-group a-horizontal a-spacing-top-micro swatches swatchesSquare'}):
    for x in i.find_all('li'):
        _.append(
            [x['data-defaultasin'], x.find(attrs={'a-size-base'}).get_text(), 'Samsung']
        )
        

col = [(a, int(re.sub("\D","",b)), c) for (a, b, c) in _[0:5]]

url = 'https://www.amazon.com/Sandisk-Ultra-Micro-UHS-I-Adapter/dp/B073K14CVB/ref=sxin_4_sxwds-bovbs?keywords=sandisk+micro+sd&pd_rd_i=B073K14CVB&pd_rd_r=1892a852-fc86-4c96-8c4f-c2d75ecb6b6e&pd_rd_w=9an3s&pd_rd_wg=GirQt&pf_rd_p=b866bc3b-cfe5-45c1-897c-a8525a3335d9&pf_rd_r=42K1ZJJN26BSQQ72YKCE&qid=1565741086&s=gateway'

soup = BeautifulSoup(request(url).text)

for i in soup.find_all(attrs={'a-unordered-list a-nostyle a-button-list a-declarative a-button-toggle-group a-horizontal a-spacing-top-micro swatches swatchesSquare'}):
    for x in i.find_all('li'):
        col.append([x['data-defaultasin'], int(re.sub("\D","", x.find(attrs={'a-size-base'}).get_text())), 'SanDisk'])

In [ ]:
col

Creating 'targets' DF

In [ ]:
targets = pd.DataFrame(data=(col), columns=['ASIN', 'cap', 'brand'])

In [ ]:
try:
    db.project.targets.insert_many(targets.to_dict('record'), ordered=False)
except:
    pass

In [ ]:
len(list(db.project.targets.find()))

### To scrape review

Creating an empty dataframe

In [9]:
df = pd.DataFrame(columns = ['ASIN', 'pg_num','review_id', 'review_score', 'vine_status', 'reviewer_name', 'review_date', 'title', 'content'])

In [ ]:
to_scrape = db.project.targets.distinct('ASIN')

to_scrape

In [10]:
to_scrape = [ 'B073JYC4XM','B073JYVKNX','B073K14CVB','B074RNRM2B','B0758NHWS8','B07MKSGZM6','B07SC7BZHR']
to_scrape.reverse()

In [17]:
for ASIN in to_scrape:
    for i in count():
        clear_output()
        if i == 0:
            continue
        _ = review_scrape(ASIN, i)
        
        rand_time = random.randint(1,5) + random.random()
        
#         print(requests.get('https://api.myip.com', proxies=proxyDict).text)
        print('---CURRENT---')
        print(ASIN)
        print('page:', i,'\nrand:', rand_time)
        print('---SCRAPED---')
        print(f'{db.project.reviews.count()}')
        display(df.groupby('ASIN').agg({'review_id': 'count'}))
        
        time.sleep(rand_time)
        if len(_) == 0:
            break
        
        try:
            db.project.reviews.insert_many(_.to_dict('record'), ordered=False)
        except:
            pass
        
        df = pd.concat([df, _])
#         display(df.groupby('ASIN').agg({'review_id': 'count'}))
        
#         with open('df.pickle','wb') as file:
#             pickle.dump(df, file)

---CURRENT---
B07SC7BZHR
page: 1 
rand: 5.737633904219872
---SCRAPED---
35667


/Users/panherbert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  from ipykernel import kernelapp as app


,review_id
ASIN,
B07MKSGZM6,100
B07SC7BZHR,6


KeyboardInterrupt: 

### To scrape questions

In [4]:
def question_scrape(ASIN, pg_num):
    # def question_scraper(ASIN, pg_num):
        # Scrape Reviews
    url = f'https://www.amazon.com/ask/questions/asin/{ASIN}/{pg_num}/ref=ask_ql_psf_ql_hza?sort=SUBMIT_DATE'

    soup = BeautifulSoup(request(url).text)

    # Stripping review_id, title and content

    links = set()

    for i in soup.findAll(True, class_ ="a-section askTeaserQuestions")[0].find_all(attrs={'a-link-normal'}):
        if i['href'] != 'javascript:void(0)':
            links.add('https://amazon.com'+i['href'])
    
    links = list(links)
    links = [i for i in links if len(i) > 30]
    links = list(filter(lambda i: i.split('/')[3] == 'ask', links))
    
    question_id = [ i.split('/')[5] for i in links ]
    
    question_dates = []
    titles = []
    question_ASIN = []

    for link in links:
        soup = BeautifulSoup(request(link).text)
        question_dates.append(soup.find(True, {'class': 'a-column a-span6 a-text-left'}).get_text().strip('asked on '))
        titles.append(soup.find(True, class_ ='a-size-large askAnswersAndComments askWrapText').span.get_text())
        question_ASIN.append(soup.find(True, class_= 'a-size-base a-link-normal')['href'].split('/')[3])

    return [{
        'question_id': a,
        'question_ASIN': b,
        'question date': c,
        'title': d
     } for (a, b, c, d) in list(zip(question_id, question_ASIN, question_dates, titles))]

In [5]:
question_scrape('B073JYC4XM', 60)

[{'question_id': 'TxBQZCRJECCEHQ',
  'question_ASIN': 'B074RNRM2B',
  'question date': 'February 25, 2019',
  'title': 'Do you think that this would work on the Galaxy S10+?'},
 {'question_id': 'Tx2RVDFJ3AEVGCA',
  'question_ASIN': 'B073JYVKNX',
  'question date': 'February 23, 2019',
  'title': 'tomameri - mp3 / mp4 player compatible?'},
 {'question_id': 'Tx2JPRQH3REEMN4',
  'question_ASIN': 'B073JYVKNX',
  'question date': 'February 25, 2019',
  'title': 'does this work in LG Realm cell phones?'},
 {'question_id': 'TxIS2OGQD0GS91',
  'question_ASIN': 'B073JYVKNX',
  'question date': 'February 24, 2019',
  'title': 'Will this work with my phone? my phone is a zte blade x'},
 {'question_id': 'TxWAVTKDQ1WKFY',
  'question_ASIN': 'B0758NHWS8',
  'question date': 'February 24, 2019',
  'title': 'Does this work with huawei mate 9? looking to buy the 400gb capacity. '},
 {'question_id': 'TxFPS2IP8J1N1T',
  'question_ASIN': 'B0758NHWS8',
  'question date': 'February 24, 2019',
  'title': "Wi

In [18]:
db.project.targets.distinct('ASIN')

['B06XWN9Q99',
 'B06XWZWYVP',
 'B06XX29S9Q',
 'B072HRDM55',
 'B073JWXGNT',
 'B073JY5T7T',
 'B073JYC4XM',
 'B073JYVKNX',
 'B073K14CVB',
 'B074RNRM2B',
 'B0758NHWS8',
 'B07MKSGZM6',
 'B07SC7BZHR']

In [ ]:
for ASIN in db.project.targets.distinct('ASIN'):
    for i in count():
        if i == 101:
            break
            
        clear_output()
        
        if i == 0:
            continue
            
        _ = question_scrape(ASIN, i)
        
        rand_time = random.randint(1,5) + random.random()
        
        print('---CURRENT---')
        print(ASIN)
        print('page:', i,'\nrand:', rand_time)
        print('---SCRAPED---')
        print(f'{db.project.questions.count()}')
        
        time.sleep(rand_time)
        if len(_) == 0:
            break
        
        try:
            db.project.questions.insert_many(_, ordered=False)
        except:
            pass
        


# temp

In [ ]:
col = []

url = 'https://www.amazon.com/Samsung-MicroSDXC-Adapter-MB-ME128GA-AM/dp/B06XWZWYVP/ref=sr_1_5?keywords=sandisk%2Bmicro%2Bsd&qid=1565739330&s=gateway&sr=8-5&th=1'

soup = BeautifulSoup(request(url).text)

_ = []

for i in soup.find_all(attrs={'a-unordered-list a-nostyle a-button-list a-declarative a-button-toggle-group a-horizontal a-spacing-top-micro swatches swatchesSquare'}):
    for x in i.find_all('li'):
        _.append(
            [x['data-defaultasin'], x.find(attrs={'a-size-base'}).get_text(), 'Samsung']
        )
        
col = _[0:5]

url = 'https://www.amazon.com/Sandisk-Ultra-Micro-UHS-I-Adapter/dp/B073K14CVB/ref=sxin_4_sxwds-bovbs?keywords=sandisk+micro+sd&pd_rd_i=B073K14CVB&pd_rd_r=1892a852-fc86-4c96-8c4f-c2d75ecb6b6e&pd_rd_w=9an3s&pd_rd_wg=GirQt&pf_rd_p=b866bc3b-cfe5-45c1-897c-a8525a3335d9&pf_rd_r=42K1ZJJN26BSQQ72YKCE&qid=1565741086&s=gateway'

soup = BeautifulSoup(request(url).text)

for i in soup.find_all(attrs={'a-unordered-list a-nostyle a-button-list a-declarative a-button-toggle-group a-horizontal a-spacing-top-micro swatches swatchesSquare'}):
    for x in i.find_all('li'):
        col.append([x['data-defaultasin'], x.find(attrs={'a-size-base'}).get_text(), 'SanDisk'])

In [ ]:
col

In [ ]:
test = pd.DataFrame(data=(col), columns=['ASIN', 'cap', 'brand'])
test

In [ ]:
from pymongo import MongoClient

In [ ]:
db = MongoClient("mongodb://@52.53.234.224", 27017)

In [ ]:
try:
    db.project.targets.insert_many(test.to_dict('record'), ordered=False)
except:
    pass

test

In [ ]:
list(db.project.targets.find())